In [6]:
import pandas as pd
prompts = pd.read_csv("../data/train_prompts.csv")
display(prompts.head())
print("Nb de prompts :", len(prompts))




,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


Nb de prompts : 2


In [2]:
from transformers import pipeline

# charge le modèle en lazy-download (~500 Mo la 1ère fois)
generator = pipeline(
    "text-generation",
    model="EleutherAI/gpt-neo-125M",
    device_map="auto",          # CPU par défaut
    framework="pt",             # PyTorch
)

def generate_essay(prompt: str, max_tokens: int = 220) -> str:
    full_prompt = f"Write a well-structured essay (~200 words) answering:\n{prompt}\nEssay:"
    out = generator(full_prompt,
                    max_new_tokens=max_tokens,
                    do_sample=True,
                    temperature=0.8,
                    top_p=0.95)[0]["generated_text"]
    # on coupe ce qui dépasse du prompt
    return out.split("Essay:")[-1].strip()


/Users/williamleroy/PycharmProjects/PythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/williamleroy/PycharmProjects/PythonProject/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps


In [7]:
print(prompts.columns)


Index(['prompt_id', 'prompt_name', 'instructions', 'source_text'], dtype='object')


In [9]:
import pandas as pd, tqdm, time

N_VARIANTS = 25          # nombre d’essais IA par sujet  (2 × 25 = 50)
ai_samples = []

for _, row in prompts.iterrows():
    prompt_text = row["instructions"]      # on se base sur l’énoncé
    for _ in range(N_VARIANTS):
        essay = generate_essay(prompt_text)
        ai_samples.append({"text": essay, "generated": 1})

ai_df = pd.DataFrame(ai_samples)
ai_df.to_csv("../data/generated_ai_essays.csv", index=False)
print("Essais IA générés :", len(ai_df))
ai_df.head()

# Supprime les textes vides ou trop courts
ai_df = ai_df[ai_df["text"].str.len() > 20]   # garde uniquement si >20 caractères
print("Après filtrage :", len(ai_df), "essais IA conservés")
ai_df.to_csv("../data/generated_ai_essays.csv", index=False)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Essais IA générés : 50
Après filtrage : 50 essais IA conservés
